# Interactive Spark Session on Synapse Compute

#### Install package (This will be updated after the offical package release)

In [ ]:
!sudo apt install libkrb5-dev -y

In [ ]:
!pip install -U azureml-synapse --index-url https://azuremlsdktestpypi.azureedge.net/sdk-release/master/588E708E0DF342C4A80BD954289657CF --extra-index-url https://pypi.python.org/simple

For JupyterLab, please additionally run:

In [ ]:
!jupyter lab build --minimize=False

__PLEASE restart kernel and then refresh web page before starting spark session.__

## 0. Magic Usage

In [ ]:
# show help
%synapse ?

## 1. Start Synapse Session

In [ ]:
# use Synapse compute linked to the NBVM's workspace.

%synapse start -c synapsecompute

In [ ]:
# use Synapse compute from anther workspace via its config file

# %synapse start -c synapse_compute_name -f config.json

In [ ]:
# use Synapse compute from anther workspace via subscription_id, resource_group and workspace_name

# %synapse start -c synapse_compute_name -s 35f16a99-532a-4a47-9e93-00305f6c40f2 -r data4mlccanary -w data4mlccanary

## 2. Data prepration

### Example-1: Read data by OpenDataset

In [ ]:
%%synapse

# 10 hottest stations in April 2020

from azureml.opendatasets import NoaaIsdWeather
from datetime import datetime
from dateutil import parser
from dateutil.relativedelta import relativedelta
from  pyspark.sql.functions import desc

start_date = parser.parse('2020-4-1')
end_date = parser.parse('2020-4-30')
isd = NoaaIsdWeather(start_date, end_date)
df = isd.to_spark_dataframe()

df.groupBy('stationName','countryOrRegion').max('temperature').orderBy(desc('max(temperature)')).show(10)

### Example-2: Read data by HDFS path

In [ ]:
%%synapse

from pyspark.sql.functions import col, desc

df = spark.read.option("header", "true").csv("wasbs://demo@dprepdata.blob.core.windows.net/Titanic.csv")
df.filter(col('Survived') == 1).groupBy('Age').count().orderBy(desc('count')).show(10)

#### 3. Session Metadata
After session started, you can check the session's metadata, find the links to Synapse portal.

In [ ]:
%synapse meta

#### 4. Stop Session
When current session reach the status timeout, dead or any failure, you must explicitly stop it before start new one. 

In [ ]:
%synapse stop